In [2]:
import sys
sys.path.append('..')

import torch
import numpy as np

from utils import ntf
from pipeline import PipelineGED

from transformers import logging
logging.set_verbosity_error()

In [18]:
checkpoints = [
    '../finetuned_models/ner_run_aug_mini/fold0/checkpoint-796/pytorch_model.bin',  
    '../finetuned_models/ner_run_aug_mini/fold1/checkpoint-796/pytorch_model.bin',  
]
model_name = 'hfl/chinese-macbert-base'


device = torch.device("cpu")
# device = torch.device("cuda:0")

data_configs = {
    'model_name':model_name,
    'maxlength':128,
    'train_val_split':-1,
    'test':True, 
    'remove_username':False,
    'remove_punctuation':False, 
    'to_simplified':False, 
    'emoji_to_text':False, 
    'split_words':False, 
    'cut_all':False, 
}
clf = PipelineGED(model_name=model_name, data_configs=data_configs)

In [24]:
texts = [
    '明天就放暑假了，班上的同学都很激动。', 
    '明天就放暑假了，班上的同学很激动。', 
]

clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


明天就放暑假了，班上的同学都很激动。
明天就放暑假了，班上的同学很激动。


array([[0.12790188, 0.8720981 ],
       [0.14710762, 0.8528924 ]], dtype=float32)

In [11]:
## 搭配
texts = [
    '女娲有一双聪明能干的手。', 
    '女娲有一双能干的手。', 
    '女娲有一双聪明的手。', 
]

print(clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True))

texts = [
    '只有恪守学术道德，才能消除学术腐败，净化学术界。', 
    '只要恪守学术道德，才能消除学术腐败，净化学术界。'
]

print(clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True))

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


女娲有一双聪明能干的手。
女娲有一双能干的手。
女娲有一双聪明的手。
[[0.31484938 0.6851507 ]
 [0.2918935  0.70810646]
 [0.2556645  0.74433553]]


d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


只有恪守学术道德，才能消除学术腐败，净化学术界。
只要恪守学术道德，才能消除学术腐败，净化学术界。
[[0.12565774 0.8743422 ]
 [0.029732   0.97026795]]


In [14]:
texts = [
    '现在高中学生不喜欢阅读世界名著的一个重要原因，是部分学生认为阅读名著不能增加高考语文得分。', 
    '学生认为阅读名著不能增加得分。', 
]

clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


现在高中学生不喜欢阅读世界名著的一个重要原因，是部分学生认为阅读名著不能增加高考语文得分。
学生认为阅读名著不能增加得分。


array([[0.0419014 , 0.9580986 ],
       [0.48035076, 0.5196492 ]], dtype=float32)

In [15]:
# 语义：单面对双面
texts = [
    '他为了民族的兴亡和人民的利益奋斗了一生。', 
    '他为了民族的复兴和人民的利益奋斗了一生。', 
]

print(clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True))

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


他为了民族的兴亡和人民的利益奋斗了一生。
他为了民族的复兴和人民的利益奋斗了一生。
[[0.78061116 0.21938887]
 [0.86008686 0.13991316]]


In [16]:
texts = [
    '从小明这一个简单的举动中，可以让人看出他是一个细心的好孩子。', 
    '小明这一个简单的举动，可以让人看出他是一个细心的好孩子。', 
]

print(clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True))


texts = [
    '我想，人是由三部分组成的：对往事的追忆、对未来的憧憬和对现时的把握。', 
    '我想，人是由三部分组成的：对往事的追忆、对现时的把握和对未来的憧憬。', 
]

print(clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True))

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


从小明这一个简单的举动中，可以让人看出他是一个细心的好孩子。
小明这一个简单的举动，可以让人看出他是一个细心的好孩子。
[[0.8812645  0.11873545]
 [0.8475716  0.15242836]]


d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


我想，人是由三部分组成的：对往事的追忆、对未来的憧憬和对现时的把握。
我想，人是由三部分组成的：对往事的追忆、对现时的把握和对未来的憧憬。
[[0.8357699  0.16423015]
 [0.86057377 0.13942622]]


In [17]:
# 成分残残缺/赘余

texts = [
    '时光的流逝不能让我淡去对故乡浓浓的思念，反之，随着年龄的增长，对故乡的思念愈发日久弥坚。', 
    '时光的流逝不能让我淡去对故乡浓浓的思念，反之，随着年龄的增长，对故乡的思念日久弥坚。', 
]

print(clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True))

texts = [
    '在《京都议定书》第二承诺期问题上，美国继续坚持称，除非中国等新兴市场大国也接受强制减排指标，否则，不接受的话，美国不可能签第二承诺期。', 
    '在《京都议定书》第二承诺期问题上，美国继续坚持称，除非中国等新兴市场大国也接受强制减排指标，否则，美国不可能签第二承诺期。', 
]

print(clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True))

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


时光的流逝不能让我淡去对故乡浓浓的思念，反之，随着年龄的增长，对故乡的思念愈发日久弥坚。
时光的流逝不能让我淡去对故乡浓浓的思念，反之，随着年龄的增长，对故乡的思念日久弥坚。
[[0.11956172 0.88043827]
 [0.10410441 0.8958956 ]]


d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


在《京都议定书》第二承诺期问题上，美国继续坚持称，除非中国等新兴市场大国也接受强制减排指标，否则，不接受的话，美国不可能签第二承诺期。
在《京都议定书》第二承诺期问题上，美国继续坚持称，除非中国等新兴市场大国也接受强制减排指标，否则，美国不可能签第二承诺期。
[[0.8552279  0.1447721 ]
 [0.8938278  0.10617216]]
